In [3]:
#30/7/24 Creates a K fold KNN model for each file in a folder provided they are Fingerprints
import gc
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '999'
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import nnmetrics
from Function_Repo import setbalance
import math
import tensorflow as tf
from tensorflow import keras
from keras.layers import BatchNormalization
import matplotlib.pyplot as plt

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Fingerprints/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' and 'organ' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df.drop(columns=['SMILES'], inplace=True)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

datasets = datasets[14:]
namelist = namelist[14:]

sets = 0
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    threshlist = []
    finalresults = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        
        
        keras.utils.set_random_seed(seed)
        tf.config.experimental.enable_op_determinism()
        length = len(xtrain.columns)
        sampcount = len(ytrain)
        temp = length
        # Encoder Creation
        enc_in = keras.Input(shape=(length, 1))
        x = keras.layers.Flatten()(enc_in)
        # Additional encoder layers
        
        layer1 = math.sqrt(3*sampcount) + 2*(math.sqrt(sampcount/(3)))
        x = keras.layers.Dense(layer1, activation="relu")(x)
        
        layer2 = math.sqrt(sampcount / 3)
        x = keras.layers.Dense(layer2, activation="relu")(x)
        
        enc_out = keras.layers.Dense(1, activation="sigmoid")(x)
        autoencoder = keras.Model(enc_in, enc_out, name='autoencoder')
        # Optimiser
        
        optimise = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)

        autoencoder.compile(optimise, loss='binary_crossentropy')
        #autoencoder.summary()
        
        ytrain = np.array(ytrain)
        epochs = 500
        batchsize = 16
        losshistory = []

        minloss = 1
        bestepoch = 0
        for epoch in range(epochs):
            verb = 0
            logs = autoencoder.fit(
            xtrain,
            ytrain,
            verbose = verb,
            epochs=1,
            batch_size=batchsize),
            validation_split=0.2,

            #autoencoder.save(str(f"models/AE-{epoch+1}.keras"))
            loss = logs[0].history['loss'][0]
            losshistory.append(loss)
            if loss < minloss:
                minloss = loss
                bestepoch = epoch

        model_list.append(autoencoder)
    
        preds = autoencoder.predict(xtest)

        epochcount = range(epochs)

        acclist = []
        thresholdlist = []
        maxval = 0
        bestthresh = 0
        for threshold in np.arange(0, 1, 0.01):
            newval = nnmetrics(preds, ytest, threshold)[12]
            acclist.append(newval)
            thresholdlist.append(threshold)
            if newval > maxval:
                maxval = newval
                bestthresh = threshold
        threshlist.append(bestthresh)
        testdat.append(maxval)
        y_values = acclist
        x_values = thresholdlist
        preds = autoencoder.predict(xvalid)
        results = nnmetrics(preds, yvalid, bestthresh)
        validlist.append(results)
        print('for fold', split + 1, 'optimal test mcc of', maxval, 'validation mcc of', results[12])

        iteration = iteration + 1

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        

    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []

    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        fp = np.asarray(fp).astype('float32')
        #appends each model's prediction to a list
        predlist = []
        for model in model_list:
            preds = model.predict(fp, verbose = 0)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        loops = 0
        for modelpreds in predlist:
            if modelpreds < threshlist[loops]:
                zercount = zercount + 1
        loops = loops + 1

        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)

    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    finalresults.append(results[12])
    del autoencoder
    gc.collect
    
    sets += 1

For dataset Ames_Mutagenicity_Morganfp.csv
19/19 [==============================] - 0s 520us/step
for fold 1 optimal test mcc of 0.5966218454213833 validation mcc of 0.5470892998920918
19/19 [==============================] - 0s 519us/step
for fold 2 optimal test mcc of 0.6587526009704956 validation mcc of 0.5727605393717712
19/19 [==============================] - 0s 550us/step
for fold 3 optimal test mcc of 0.6148014511553934 validation mcc of 0.5628421478148414
19/19 [==============================] - 0s 485us/step
for fold 4 optimal test mcc of 0.5983319979898829 validation mcc of 0.5451613502165782
19/19 [==============================] - 0s 509us/step
for fold 5 optimal test mcc of 0.5740837649307148 validation mcc of 0.5782036061910832

validation metrics of:
positives in data 839
negatives in data 658
net accuracy = 0.8102872411489646
mcc = 0.6183520659265372
For dataset Respiratory_Toxicity_Morganfp.csv
4/4 [==============================] - 0s 560us/step
for fold 1 optimal te

4/4 [==============================] - 0s 776us/step
for fold 5 optimal test mcc of 0.5727443617941211 validation mcc of 0.5507479070771424

validation metrics of:
positives in data 70
negatives in data 240
net accuracy = 0.8225806451612904
mcc = 0.4385666810427817
For dataset Developmental_Toxicity_Morganfp.csv
1/1 [==============================] - 0s 9ms/step
for fold 1 optimal test mcc of 0.25 validation mcc of 0
1/1 [==============================] - 0s 9ms/step
for fold 2 optimal test mcc of 0.16666666666666666 validation mcc of 0.3333333333333333
1/1 [==============================] - 0s 9ms/step
for fold 3 optimal test mcc of 0 validation mcc of 0
1/1 [==============================] - 0s 12ms/step
for fold 4 optimal test mcc of 0 validation mcc of 0
1/1 [==============================] - 0s 10ms/step
for fold 5 optimal test mcc of 0.16666666666666666 validation mcc of 0

validation metrics of:
positives in data 8
negatives in data 6
net accuracy = 0.7142857142857143
mcc = 0.45

In [2]:
namelist

['Ames_Mutagenicity_Morganfp.csv',
 'Respiratory_Toxicity_Morganfp.csv',
 'SR-HSE_Morganfp.csv',
 'NR-aromatase_Morganfp.csv',
 'SR-p53_Morganfp.csv',
 'Cardiotoxicity-1_Morganfp.csv',
 'Developmental_Toxicity_Morganfp.csv',
 'Cardiotoxicity-30_Morganfp.csv']

In [3]:
temp = pd.DataFrame(finalresults, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Data/Model_Outputs/Fingerprint/NeuralNet_Fingerprint_Results')
resultframe

,Dataset,MCC
0,CarcinogenicityMorganfp.csv,0.0
1,morganfp_SR-p53.csv,NaN
2,morganfp_NR-AhR.csv,NaN
3,morganfp_NR-ER-LBD.csv,NaN
4,mutagenTTorsionfp.csv,NaN
5,morganfp_NR-PPAR-gamma.csv,NaN
6,Respiratory_ToxicityMorganfp.csv,NaN
7,mutagenMACCSfp.csv,NaN
8,Reproductive_ToxicityTTorsionfp.csv,NaN
9,CarcinogenicityAvafp.csv,NaN


In [14]:
bing = namelist[86:]

In [15]:
bing

['Cardiotoxicity-30_Morganfp.csv', 'Carcinogenicity_MACCSfp.csv']

In [11]:
len(namelist)

88